# Strategic Finance Decision-Making and Optimization Solutions

## Author: Sally Lee

In [1]:
using PyPlot, JuMP, GLPK, DataFrames, CSV

## Short-term Financing Problem

The company has two methods to conduct short-term financing. It can borrow up to $100k monthly at a 0.5% interest rate per month. It can also issue a 3-month zero-coupon bond at 1.5% for a three month period. Any available cash at the end of each month can earn 0.2% interest rate per month.

The company's net cash flow is as described in the table below
|Month	| Jul 2013	| Aug 2013	| Sep 2013	| Oct 2013	| Nov 2013	| Dec 2013	
| ------| ------| ------| ------| -------| -------| -------
| NCF	| -250	| -50	| 225	| -175 | 150 | 400

We set our variables as following:

- Lending credit as xi - 0.5% borrowing cost
- Bond issuance as yi - 1.5% borrowing cost
- Excess cash as zi - 0.2% return on cash
- v as objective function variable - our goal is to maximise v (v is the same as  z6 - excess cash in December 2013)

The net cash flow (NCF) requirement for each month serves as a constraint in our optimization problem. 

#### Our objective function and constraints are defined as follows:

In [2]:
using JuMP, GLPK

solver = JuMP.Model(GLPK.Optimizer)

# Set the variables
@variable(solver, 0 <= x[1:5] <= 100) # amount borrowed from credit in month i
@variable(solver, 0 <= y[1:3]) # bond issued in month i
@variable(solver, 0 <= z[1:5]) # excess cash in month i
@variable(solver, v >= 0) # company wealth in Dec

# Set the constraints for cash flows each month
@constraint(solver, x[1] + y[1] - z[1] == 250)   # July
@constraint(solver, x[2] + y[2] - 1.005*x[1] + 1.002*z[1] - z[2] == 50)    # August
@constraint(solver, x[3] + y[3] - 1.005*x[2] + 1.002*z[2] - z[3] == -225)  # September
@constraint(solver, x[4] - 1.015*y[1] - 1.005*x[3] + 1.002*z[3] - z[4] == 175) # October
@constraint(solver, x[5] - 1.015*y[2] - 1.005*x[4] + 1.002*z[4] - z[5] == -150) # November
@constraint(solver, -1.015*y[3] - 1.005*x[5] + 1.002*z[5] - v == -400)     # December

# Objective is to maximaize the cash flow in Dec
@objective(solver, Max, v)

optimize!(solver)

print(solver)

println()
println(termination_status(solver))
println("Objective value: ", JuMP.objective_value(solver))
println("x = ", value.(x))
println("y = ", value.(y))
println("z = ", value.(z))


OPTIMAL
Objective value: 294.4028592065869
x = [100.0, 100.0, 0.0, 100.0, 1.7574999999999648]
y = [150.0, 50.499999999999986, 102.2964071856287]
z = [0.0, 0.0, 226.7964071856287, 0.0, 0.0]


### First, we find the amount of borrowing between the two instruments for each month that maximizes the total amount of cash available in December 2013.

In [3]:
# Try to use metrics to solve the problem - set x1 to x14 to represent xi, yi, and zi
solver = JuMP.Model(GLPK.Optimizer)

# Set matrices - A for constraint coefficient, b for constraint solutions, c for objective function 
A = [ 1.000  0.000  0.000  0.000  0.000  1.000  0.000  0.000 -1.000  0.000  0.000  0.000  0.000  0.000;
         -1.005  1.000  0.000  0.000  0.000  0.000  1.000  0.000  1.002 -1.000  0.000  0.000  0.000  0.000;
          0.000 -1.005  1.000  0.000  0.000  0.000  0.000  1.000  0.000  1.002 -1.000  0.000  0.000  0.000;
          0.000  0.000 -1.005  1.000  0.000 -1.015  0.000  0.000  0.000  0.000  1.002 -1.000  0.000  0.000;
          0.000  0.000  0.000 -1.005  1.000  0.000 -1.015  0.000  0.000  0.000  0.000  1.002 -1.000  0.000;
          0.000  0.000  0.000  0.000 -1.005  0.000  0.000 -1.015  0.000  0.000  0.000  0.000  1.002 -1.000;]
b = [250.0; 50.0; -225.0; 175.0; -150.0; -400.0;]    
c = [0.000; 0.000; 0.000; 0.000; -1.005; 0.000; 0.000; -1.015; 0.000; 0.000; 0.000; 0.000; 1.002; -1.000;]


@variable(solver,  x[1:14] >= 0)

@constraint(solver, x[1] <= 100)
@constraint(solver, x[2] <= 100)
@constraint(solver, x[3] <= 100)
@constraint(solver, x[4] <= 100)
@constraint(solver, x[5] <= 100)
@constraint(solver, A * x .== b)

@objective(solver, Max, x[14])

status = optimize!(solver)

println()
println(termination_status(solver))
println("Objective value: ", objective_value(solver))
println("x1 = ", value(x[1]))
println("x2 = ", value(x[2]))
println("x3 = ", value(x[3]))
println("x4 = ", value(x[4]))
println("x5 = ", value(x[5]))
println("y1 = ", value(x[6]))
println("y2 = ", value(x[7]))
println("y3 = ", value(x[8]))
println("z1 = ", value(x[9]))
println("z2 = ", value(x[10]))
println("z3 = ", value(x[11]))
println("z4 = ", value(x[12]))
println("z5 = ", value(x[13]))
println("v = ", value(x[14]))



OPTIMAL
Objective value: 294.4028592065869
x1 = 100.00000000000001
x2 = 100.00000000000001
x3 = 0.0
x4 = 100.0
x5 = 1.7574999999999759
y1 = 149.99999999999997
y2 = 50.499999999999986
y3 = 102.2964071856287
z1 = 0.0
z2 = 0.0
z3 = 226.79640718562868
z4 = 0.0
z5 = 0.0
v = 294.4028592065869


#### Credit/Bond borrowing per month (in thousands of dollars):

In [4]:
dfFin = DataFrame([
        ["Jul", "Aug", "Sep", "Oct", "Nov", "Dec"],
        [100, 100, 0, 100, 1.76, 0], 
        [150, 50.50, 102.30, 0, 0, 0]
        ], 
        [:Month, :Credit, :Bonds])

Row,Month,Credit,Bonds
,String,Float64,Float64
1,Jul,100.0,150.0
2,Aug,100.0,50.5
3,Sep,0.0,102.3
4,Oct,100.0,0.0
5,Nov,1.76,0.0
6,Dec,0.0,0.0


### Then, we try to find the optimal amount of cash on Dec 2013.

The optimal amount of cash on Dec 2013 is around 294,402.86 dollar.

### FInally, we interpret the results and develop a storyline implied by the LP solution.

The LP solution provides a strategic plan for optimizing the total amount of cash available in December 2013 through borrowing between credit and bonds over a six-month period.

In July and August, the corporation experienced negative cash flows, requiring financing from both credit and bonds to cover the cash outflows.

By September, the cash flow turned positive, allowing the corporation to repay the credit borrowing while still borrowing from bonds to supplement cash reserves.

In October, the corporation experienced another period of negative cash flows and thus borrowed from credit to meet financial obligations.

In November, since the cash flow remained positive, the corporation only borrowed a small amount from credit, indicating a strategic decision to minimize borrowing and maintain a conservative financial approach.

In December, the corporation's cash flow significantly improved, enabling it to avoid borrowing from either instrument. Instead, it generated excess cash, which is available for investment to accumulate interest, further enhancing the corporation's financial position.

Overall, the linear programming solution demonstrates a dynamic approach to cash management by leveraging borrowing options to navigate through periods of negative cash flows while minimizing borrowing costs during periods of positive cash flows. This strategy effectively optimized cash availability, leading to a favorable financial outcome by December 2013.

## Currency Arbitrage Problem

The table below presents hypothetical bid/ask quotes for four major currencies at a given date/time:

| Pair	    | Bid	    | Ask
| ------    | ------    | ------
| EUR/USD	| 0.87060	| 0.87063
| GBP/USD	| 1.42790	| 1.42796	
| USD/JPY	| 133.330	| 133.333	
| EUR/GBP	| 0.60970	| 0.60972	
| EUR/JPY	| 116.140	| 116.144	
| GBP/JPY	| 190.480	| 190.476

The first currency in a currency pair is called the base currency. The bid and the ask are always quoted in terms of the base currency. For example, if you are buying on EUR/USD trade, then you are effectively receiving EUR in exchange for USD at the ask quote.

We were given USD 1 to attempt an arbitrage trade, assume no transaction costs.

### We first write down a linear programming problem to buy and sell quantities of the base currencies and arbitrage/profit of at most USD 10k.

We set the problem by setting variables related to all exchanges between currencies.

D = quantity of USD generated through arbitrage

DE, DP, DY = quantity of USD changed into EUR, GBP, and JPY respectively

ED, EP, EY = quantity of EUR changed into USD, GBP, and JPY respectively

PD, PE, PY = quantity of GBP changed into USD, EUR, and JPY respectively

YD, YE, YP = quantity of JPY changed into USD, EUR, and GBP respectively

We also transformed the bid/ask quote into exchange rate. The following table shows the exchange rates that could be use as our constraints.

| TO / FROM	| USD	| EUR	| JPY	| GBP
| ----------| ------| ------| ------| -------
| USD	| 1	| 0.87060	| 1/133.333	| 1.42790
| EUR	| 1/0.87063	| 1	| 1/116.144	| 1/0.60972
| JPY	| 133.330	| 116.140	| 1	| 190.480
| GBP	| 1/1.42796	| 0.60970	| 1/190.476	| 1

In [5]:
using JuMP, GLPK

solver = JuMP.Model(GLPK.Optimizer)

# Set the variables
@variable(solver, DE >= 0) # quantity of USD to EUR
@variable(solver, DP >= 0) # quantity of USD to GBP
@variable(solver, YD >= 0) # quantity of JPY to USD
@variable(solver, PE >= 0) # quantity of GBP to EUR
@variable(solver, YE >= 0) # quantity of JPY to EUR
@variable(solver, YP >= 0) # quantity of JPY to GBP
@variable(solver, ED >= 0) # quantity of EUR to USD
@variable(solver, PD >= 0) # quantity of GBP to USD
@variable(solver, DY >= 0) # quantity of USD to JPY
@variable(solver, EP >= 0) # quantity of EUR to GBP
@variable(solver, EY >= 0) # quantity of EUR to JPY
@variable(solver, PY >= 0) # quantity of GBP to JPY
@variable(solver, D >= 0) # quantity of USD generated through arbitrage

# Set the constraints for cash flows each month
@constraint(solver, D + DE + DP + DY - 0.87060*ED - 1.42790*PD - (1/133.333)*YD == 1)   # USD
@constraint(solver, ED + EP + EY - (1/0.87063)*DE - (1/0.60972)*PE - (1/116.144)*YE == 0)    # EUR
@constraint(solver, PD +PE + PY - (1/1.42796)*DP - 0.60970*EP - (1/190.476)*YP == 0)  # GBP
@constraint(solver, YD + YE + YP - 133.330*DY - 116.140*EY - 190.480*PY == 0) # JPY
@constraint(solver, D <= 10000) # USD arbitrage

# Objective is to maximaize the cash flow in Dec
@objective(solver, Max, D)

optimize!(solver)

print(solver)

### Then, we solve for the buy/sell quantities and the arbitrage/profit amount.

In [6]:
println()
println(termination_status(solver))
println("Objective value: ", JuMP.objective_value(solver))
println("DE = ", value.(DE))
println("DP = ", value.(DP))
println("YD = ", value.(YD))
println("PE = ", value.(PE))
println("YE = ", value.(YE))
println("YP = ", value.(YP))
println("ED = ", value.(ED))
println("PD = ", value.(PD))
println("DY = ", value.(DY))
println("EP = ", value.(EP))
println("EY = ", value.(EY))
println("PY = ", value.(PY))


OPTIMAL
Objective value: 10000.0
DE = 2.0620299945404414e7
DP = 0.0
YD = 2.7506996492876062e9
PE = 0.0
YE = 0.0
YP = 0.0
ED = 0.0
PD = 0.0
DY = 0.0
EP = 0.0
EY = 2.368434345864996e7
PY = 0.0


The arbitrage amount is 10,000 and the buy/sell quantities among currencies are as following (rounded to the 2nd decimal point):

| TO / FROM	| USD	| EUR	| JPY	| GBP
| ----------| ------| ------| ------| -------
| USD	| 0	| 0	| 2,750,699,649.29	| 0
| EUR	| 20,620,299.95	| 0	| 0	| 0
| JPY	| 0	| 23,684,343.46	| 0	| 0
| GBP	| 0	| 0	| 0	| 0

### Finally, we interpret the results and develop a story of the trade executions and the resulting profit.

The above results suggest that the optimal trade executions will be:
1. use all available USD to buy EUR
2. use all available EUR to buy JPY
3. use all available JPY to buy USD
4. Repeat the above steps until the maximum arbitrage of USD 10,000 is met

The result also demonstrates that the trade execution can yield infinite arbitrage value in the absence of transaction costs, limits on transactions, or maximum profit constraints. However, real-world currency exchange typically incurs high transaction fees. Incorporating fees and other costs into the optimization problem may offer a more realistic solution for practical applications in real-world scenarios.